In [1]:
import import_ipynb
# 원하는 모델로 바꾸어 사용하기
from vision_model_final import Mini_Xception,NN

import cv2
from PIL import ImageFont, ImageDraw, Image
import numpy as np
import torch
import torchvision.transforms as transforms

import os

importing Jupyter notebook from vision_model_final.ipynb


In [2]:
# (주의) OpenCV는 Colab에서 제대로 실행되지 않음.
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [3]:
# 사용할 모델 선언하기
NN = NN().to(device)

# train이 아닌, evaluation 과정
NN.eval()

# 기존에 학습한 모델 불러오기. XXXXXXXXXXXX에 epoch 번호 작성.
path = './checkpoint/model_weigths1_NN/weights_epoch_' + '144' + '.pth.tar'
checkpoint = torch.load(path, map_location=device)
NN.load_state_dict(checkpoint['mini_xception'])

# Face detection을 위한 CascadeClassifier 모델 불러오기
path = "haarcascade_frontalface_default.xml"
face_detector = cv2.CascadeClassifier(path)

In [4]:
# Face detection을 위한 DNN 모델
# Abstract class / Interface
class FaceDetectorIface:
    def detect_faces(self, frame):
        raise NotImplementedError

class DnnDetector(FaceDetectorIface):
    """
        SSD (Single Shot Detectors) based face detection (ResNet-18 backbone(light feature extractor))
    """
    def __init__(self, root=None):
        self.prototxt = "deploy.prototxt.txt"
        self.model_weights = "res10_300x300_ssd_iter_140000.caffemodel"

        if root:
            self.prototxt = os.path.join(root, self.prototxt)
            self.model_weights = os.path.join(root, self.model_weights)

        self.detector = cv2.dnn.readNetFromCaffe(prototxt=self.prototxt, caffeModel=self.model_weights)
        self.threshold = 0.5 # to remove weak detections

    def detect_faces(self,frame):
        h = frame.shape[0]
        w = frame.shape[1]

        # required preprocessing(mean & variance(scale) & size) to use the dnn model
        """
            Problem of not detecting small faces if the image is big (720p or 1080p)
            because we resize to 300,300 ... but if we use the original size it will detect right but so slow
        """
        resized_frame = cv2.resize(frame, (300, 300))
        blob = cv2.dnn.blobFromImage(resized_frame, 1.0, resized_frame.shape[0:2], (104.0, 177.0, 123.0))
        # detect
        self.detector.setInput(blob)
        detections = self.detector.forward()
        faces = []
        # shape 2 is num of detections
        for i in range(detections.shape[2]):
            confidence = detections[0,0,i,2] 
            if confidence < self.threshold:
                continue

            # model output is percentage of bbox dims
            box = detections[0, 0, i, 3:7] * np.array([w, h, w, h])
            box = box.astype("int")
            (x1,y1, x2,y2) = box

            # x,y,w,h
            faces.append((x1,y1,x2-x1,y2-y1))
            # print(confidence)
        return faces

In [5]:
face_detector_dnn = DnnDetector()

In [6]:
def sort_vec(vec):
    """
    7차원 감정 벡터의 순서를 일치시키기 위한 함수
        
    매개변수 (Parameters)
    ----------
    vec : numpy.ndarray, shape-(7,)
        vision 모델의 출력값인 7차원 벡터.
        fer2013 dataset의 emotion label의 순서대로 구한 감정 벡터. 
        
    반환 값 (Returns)
    -------
    numpy.ndarray, shape-(7,)
        NLP dataset의 emotion index의 순서대로 재정렬한 감정 벡터
    
    참고사항
    -------
    (fer2013 dataset)
        0: 'Angry',
        1: 'Disgust', 
        2: 'Fear', 
        3: 'Happy', 
        4: 'Sad', 
        5: 'Surprise', 
        6: 'Neutral'
        
    (NLP dataset)
        0: 'Fear',
        1: 'Surprise', 
        2: 'Angry', 
        3: 'Sad', 
        4: 'Neutral', 
        5: 'Happy', 
        6: 'Disgust'  
    """
    ## 여기에 코드 작성
    
    return np.array([vec[2], vec[5], vec[0], vec[4], vec[6], vec[3], vec[1]])

In [7]:
def get_label_emotion(label):
    """
    label 값에 대응되는 감정의 이름 문자열을 구하기 위한 함수
        
    매개변수 (Parameters)
    ----------
    label : int
        emotion label 번호
        
    반환 값 (Returns)
    -------
    String
        label 번호에 대응되는 감정의 이름 문자열
    
    참고사항
    -------
    (NLP dataset)
        0: 'Fear',
        1: 'Surprise', 
        2: 'Angry', 
        3: 'Sad', 
        4: 'Neutral', 
        5: 'Happy', 
        6: 'Disgust'      
    """
    ## 여기에 코드 작성

    data = {0: 'Fear',
            1: 'Surprise',
            2: 'Angry',
            3: 'Sad',
            4: 'Neutral',
            5: 'Happy',
            6: 'Disgust'}
    
    return data[label]

In [8]:
def cos_sim(A, B):
    """
    두 벡터 A, B의 코사인 유사도를 구하기 위한 함수
        
    매개변수 (Parameters)
    ----------
    A : numpy.ndarray, shape-(N,)
    B : numpy.ndarray, shape-(N,)
        
    반환 값 (Returns)
    -------
    numpy.float64
        두 벡터 A, B의 코사인 유사도 값      
    """
    ## 여기에 코드 작성
    return np.dot(A, B) / (np.linalg.norm(A) * np.linalg.norm(B))

In [9]:
def predict_video(nlp_vec, sentence):
    """
    웹캠을 통해 받아온 실시간 영상 속에서
    1) CascadeClassifier (혹은 다른 모델)을 통해 얼굴을 탐지하고
    2) Mini_Xception (혹은 다른 모델)을 통해 얼굴 표정으로부터 7차원 감정 벡터를 추출하여
    3) sort_vec 함수를 통해 2)에서 구한 감정 벡터의 순서를 재정렬한 후
    4) cos_sim 함수를 이용하여 입력받은 문장에서 추출한 7차원 감정 벡터와의 코사인 유사도를 계산.
    5) OpenCV 라이브러리를 사용하여, 문장과 표정에서 추출한 각 감정, 그리고 표정 연기에 대한 점수(코사인 유사도)를 window 상에 시각화.
    
    매개변수 (Parameters)
    ----------
    nlp_vec : 입력한 문장에서 추출한 7차원 감정 벡터
    sentence : 사용자가 표정 연기 연습의 목적으로 입력한 문장
    """
    
    # OpenCV 실시간 웹캠 영상 불러오기
    cap = cv2.VideoCapture(0)

    # 프레임의 사이즈 계산 (height, width 구하기)
    ## 여기에 코드 작성
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    
    size = (width, height)
    
    font = cv2.FONT_HERSHEY_SIMPLEX
    font_kor = ImageFont.truetype("malgun.ttf", 18)
    
    softmax = torch.nn.Softmax()
    
    while cap.isOpened():
        ret, frame = cap.read()
        
        if ret:
            # 문장의 최대 확률 감정
            emotion_max = np.argmax(nlp_vec)
            nlp_percentage = np.round(nlp_vec[emotion_max], 2)
            nlp_emotion_label = get_label_emotion(emotion_max)
            
            # 한글 문장 출력을 위한 하단 색 띠 만들기
            frame[height-70:height, 0:width] = (50, 100, 50)
            frame_pil = Image.fromarray(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))

            
            # 한글 문장 출력을 위한 PIL 라이브러리 사용
            ## 여기에 코드 작성
            draw = ImageDraw.Draw(frame_pil)
            
            text = "emotion : " + nlp_emotion_label + " (" + str(nlp_percentage) + ")"
            draw.text(((width // 2), height - 46),
                      text,
                      font=font_kor, fill=(255,255,255), anchor="mm")
            
            text = "\"" + sentence + "\""
            draw.text(((width // 2), height - 23),
                      text,
                      font=font_kor, fill=(255,255,255), anchor="mm")

            # 한글 문장이 출력되어 있는 프레임, frame_GUI
            frame_GUI = cv2.cvtColor(np.array(frame_pil), cv2.COLOR_RGB2BGR)
            
            # CascadeClassifier 이용한 얼굴 탐지 (faces : 얼굴 탐지 결과 얻어진, face(x,y,w,h)로 이루어진 sequential data)
            ## 여기에 코드 작성
            faces = face_detector_dnn.detect_faces(frame)
            
            for face in faces:
                
                (x,y,w,h) = face
            
                # 웹캠에서 인식한 얼굴을 모델에 넣어주기 위한 전처리
                '''
                전처리 후, input_face에 저장
                 1) face의 좌표에 따라 얼굴 부분 프레임만 잘라내기
                 2) BGR2GRAY로 흑백 변환하기
                 3) (48,48)로 resize
                 4) 히스토그램 평활화 적용
                 5) Tensor로 바꾸고 device에 저장
                 6) (1,48,48)로 차원 증가
                '''
                ## 여기에 코드 작성
                face_frame = frame[y:y+h, x:x+w]
                
                face_frame = cv2.cvtColor(face_frame, cv2.COLOR_BGR2GRAY)
                face_frame = cv2.resize(face_frame, (48, 48))
                face_frame = cv2.equalizeHist(face_frame)
                
                face_frame = face_frame / 255
                
                input_face = torch.tensor(face_frame)
                input_face = input_face.to(device)
                input_face = input_face.reshape(1, 1, 48, 48)

                with torch.no_grad():
                    # 모델 출력값의 shape : [1, 7, 1, 1]
                    emotion_vec = NN(input_face.float()).squeeze()
                    
                    # 7차원 감정 확률 벡터
                    vision_vec = softmax(emotion_vec)
                    vision_vec = vision_vec.cpu().detach().numpy() # .reshape(-1,1)
                    
                    vision_vec = sort_vec(vision_vec)
                    
                    # 코사인 유사도 점수
                    ## 여기에 코드 작성
                    score = cos_sim(vision_vec, nlp_vec)

                    # GUI 상에서 출력할 정보
                    '''
                     1) 한글 문장과 최대 확률 감정, 그 확률 (이미 PIL 라이브러리로 해결)
                     2) 코사인 유사도 점수, score
                     3) 표정의 최대 확률 감정과 그 확률
                    '''
                    ## 여기에 코드 작성
                    vision_emotion_label = get_label_emotion(np.argmax(vision_vec))
                    vision_percentage = np.max(vision_vec)
                    
                    # 얼굴 표정 주변의 정보 출력을 위한 OpenCV 라이브러리 사용
                    ## 여기에 코드 작성
                    cv2.rectangle(frame_GUI, (x,y), (x+w,y+h), (255,0,0), 2)
                    cv2.rectangle(frame_GUI, (x,y), (x+w,y-70), (0,0,0), -1)
                    cv2.putText(frame_GUI,"Score: [{:2.1f}""]".format(score * 100),
                                (x, y - 46),font
                                ,0.6, (0,255,255))
                    cv2.putText(frame_GUI,
                                str(vision_emotion_label),
                                (x, y - 23),font,
                                0.4, (0, 255, 255))
                    cv2.putText(frame_GUI,
                                "[{:.2f}]".format(vision_percentage),
                                (x + w // 2 + 10, y - 23), font,
                                0.4, (255,255,0))
                    
                    
            cv2.imshow("Video", frame_GUI)
            # 탈출 조건 : esc ( OxFF==27 )
            if cv2.waitKey(1) & 0xff == 27:
                break
            
        else:
            break
    
    # 종료
    cap.release()
    cv2.destroyAllWindows()